In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os

In [ ]:
class CFG:
    data_path = './../data/bank_statements'
    column_names = ['Date', 'Transaction Value', 'Narration', 'Account Balance']
    fig_size = (15, 9)
    n_rows = 20
    
# todo: table formatting, float formatting, date formatting

In [ ]:
# categorize transactions into Income and Expenses
def categorize_transaction(narration):  # fix_me
    if 'credit' in narration.lower():
        return 'Income'
    else:
        return 'Expense'

# Brisbane

In [ ]:
brisbane_df = pd.read_csv(
    os.path.join(CFG.data_path, 'Brisbane.csv'),
    names=CFG.column_names
)
brisbane_df.head(CFG.n_rows)

In [ ]:
brisbane_df['Date'] = pd.to_datetime(brisbane_df['Date'], dayfirst=True, errors='coerce')
brisbane_df.dtypes

In [ ]:
# display basic information about the dataset
brisbane_df.info()

In [ ]:
# summary statistics
brisbane_df.describe()

In [ ]:
# check for missing values
brisbane_df.isnull().sum()

In [ ]:
brisbane_df['Category'] = brisbane_df['Narration'].apply(categorize_transaction)  # fix_me
brisbane_df.head(CFG.n_rows)  # fix_me 

In [ ]:
# sort by date to ensure correct sequential processing
brisbane_df = brisbane_df.sort_values(by='Date').reset_index(drop=True)

In [ ]:
# calculate the change in account balance for each transaction
brisbane_df['Balance Change'] = brisbane_df['Account Balance'].diff()

In [ ]:
# classify transactions based on the change in account balance
brisbane_df['Accounting Category'] = brisbane_df['Balance Change'].apply(lambda x: 'Credit' if x > 0 else 'Debit')
brisbane_df.head(CFG.n_rows)

In [ ]:
# monthly summary of transactions
brisbane_monthly_summary = brisbane_df.groupby([brisbane_df['Date'].dt.to_period('M'), 'Accounting Category'])['Transaction Value'].sum().unstack().fillna(0)

In [ ]:
# calculate net balance for each month
brisbane_monthly_summary['Net Cash'] = brisbane_monthly_summary['Credit'] + brisbane_monthly_summary['Debit']

In [ ]:
# calculate total income, total expenses, and net balance
total_income = brisbane_df[brisbane_df['Accounting Category'] == 'Credit']['Transaction Value'].sum()
total_expenses = brisbane_df[brisbane_df['Accounting Category'] == 'Debit']['Transaction Value'].sum()
net_balance = total_income + total_expenses

In [ ]:
# display monthly summary
brisbane_monthly_summary
# todo: add trend line

In [ ]:
# Print key metrics
print(f"Total Income: {total_income}")
print(f"Total Expenses: {total_expenses}")
print(f"Net Balance: {round(net_balance, 2)}")

In [ ]:
plt.figure(figsize=CFG.fig_size)
plt.plot(brisbane_df['Date'], brisbane_df['Account Balance'], marker='o', linestyle='-')
plt.title('Brisbane Account Balance')
plt.xlabel('Date')
plt.ylabel('Account Balance')
plt.grid(True)
plt.show()

In [ ]:
# plotting monthly income and expenses with net cash line
fig, ax1 = plt.subplots(figsize=CFG.fig_size)

brisbane_monthly_summary[['Credit', 'Debit']].plot(kind='bar', stacked=True, ax=ax1)
ax1.set_title('Monthly Trading')
ax1.set_xlabel('Month')
ax1.set_ylabel('Amount')
ax1.grid(True)

# plot net cash line
ax2 = ax1.twinx()
ax2.plot(brisbane_monthly_summary.index.astype(str), brisbane_monthly_summary['Net Cash'], color='red', marker='o', linestyle='-', label='Net Cash')
ax2.set_ylabel('Net Cash')

# combine legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left')
plt.show()

In [ ]:
# detect anomalies in transaction values
brisbane_df['Transaction Value'].plot(kind='box', figsize=CFG.fig_size)
plt.title('Distribution of Transaction Values')
plt.ylabel('Transaction Value')
plt.show()
# todo: boxplots for each month

In [ ]:
# interactive box plot using plotly
fig = px.box(brisbane_df, y='Transaction Value', points="all", hover_data=['Date', 'Narration', 'Transaction Value'])
fig.update_layout(
    title='Distribution of Transaction Values',
    yaxis_title='Transaction Value'
)
fig.show()

In [ ]:
# Identify "Square" transactions and create a new column
brisbane_df['Narration Group'] = brisbane_df['Narration'].apply(lambda x: 'Square' if 'square' in x.lower() else x)
# brisbane_df.head(CFG.n_rows)

In [ ]:
# categorize transactions into Credit and Debit based on account balance changes
brisbane_df = brisbane_df.sort_values(by='Date').reset_index(drop=True)
brisbane_df['Balance Change'] = brisbane_df['Account Balance'].diff()
brisbane_df['Category'] = brisbane_df['Balance Change'].apply(lambda x: 'Credit' if x > 0 else 'Debit')
# brisbane_df.head(CFG.n_rows)

In [ ]:
# filter out negative values and select only expenses
expense_df = brisbane_df[brisbane_df['Category'] == 'Debit']
# expense_df.head(CFG.n_rows)

In [ ]:
income_df = expense_df[expense_df['Transaction Value'] > 0]
# income_df.head(CFG.n_rows)

In [ ]:
# group by 'Narration Group' and sum the 'Transaction Value'
income_breakdown = income_df.groupby('Narration Group')['Transaction Value'].sum()
# income_breakdown.head(CFG.n_rows)

In [ ]:
# get the top 10 expense categories and group the rest into 'Other'
top_income_breakdown = income_breakdown.nlargest(10)
other_expenses = income_breakdown.sum() - top_income_breakdown.sum()
top_income_breakdown['Other'] = other_expenses

In [ ]:
# plot expense breakdown
top_income_breakdown.plot(kind='pie', autopct='%1.1f%%', figsize=CFG.fig_size)
plt.title('Top 10 Income Breakdown')
plt.ylabel('')
plt.show()

# Southside

In [ ]:
southside_df = pd.read_csv(
    os.path.join(CFG.data_path, 'Southside.csv'),
    names=CFG.column_names
)
southside_df.head(CFG.n_rows)

In [ ]:
southside_df['Date'] = pd.to_datetime(
    southside_df['Date'], 
    dayfirst=True, 
    errors='coerce'
)
southside_df.dtypes

In [ ]:
southside_df.describe()

In [ ]:
southside_df.info()

In [ ]:
southside_df.isnull().sum()

In [ ]:
southside_df['Category'] = southside_df['Narration'].apply(categorize_transaction)
southside_df.head(CFG.n_rows)  # fix_me

In [ ]:
# sort by date to ensure correct sequential processing
southside_df = southside_df.sort_values(by='Date').reset_index(drop=True)

In [ ]:
# calculate the change in account balance for each transaction
southside_df['Balance Change'] = southside_df['Account Balance'].diff()

In [ ]:
# classify transactions based on the change in account balance
southside_df['Category'] = southside_df['Balance Change'].apply(lambda x: 'Credit' if x > 0 else 'Debit')
southside_df.head(CFG.n_rows)  # fix_me

In [ ]:
# monthly summary of transactions
monthly_summary = southside_df.groupby([southside_df['Date'].dt.to_period('M'), 'Category'])['Transaction Value'].sum().unstack().fillna(0)

In [ ]:
# calculate net balance for each month
monthly_summary['Net Cash'] = monthly_summary['Credit'] + monthly_summary['Debit']

In [ ]:
# Calculate total income, total expenses, and net balance
total_income = southside_df[southside_df['Category'] == 'Credit']['Transaction Value'].sum()
total_expenses = southside_df[southside_df['Category'] == 'Debit']['Transaction Value'].sum()
net_cash = total_income + total_expenses

In [ ]:
# Print key metrics
print(f"Total Income: {total_income}")
print(f"Total Expenses: {total_expenses}")
print(f"Net Cash: {round(net_cash, 2)}")

In [ ]:
monthly_summary

In [ ]:
plt.figure(figsize=CFG.fig_size)
plt.plot(southside_df['Date'], southside_df['Account Balance'], marker='o', linestyle='-')
plt.title('Southside Account Balance')
plt.xlabel('Date')
plt.ylabel('Account Balance')
plt.grid(True)
plt.show()
# todo: add trend line

In [ ]:
# plotting monthly income and expenses with net cash line
fig, ax1 = plt.subplots(figsize=CFG.fig_size)

monthly_summary[['Credit', 'Debit']].plot(kind='bar', stacked=True, ax=ax1)
ax1.set_title('Monthly Trading')
ax1.set_xlabel('Month')
ax1.set_ylabel('Amount')
ax1.grid(True)

# Plot net cash line
ax2 = ax1.twinx()
ax2.plot(monthly_summary.index.astype(str), monthly_summary['Net Cash'], color='red', marker='o', linestyle='-', label='Net Cash')
ax2.set_ylabel('Net Cash')

# Combine legends
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left')

plt.show()

In [ ]:
# Detect anomalies in transaction values MPL
southside_df['Transaction Value'].plot(kind='box', figsize=CFG.fig_size)
plt.title('Distribution of Transaction Values')
plt.ylabel('Transaction Value')
plt.show()

In [ ]:
fig = px.box(southside_df, y='Transaction Value', points="all", hover_data=['Date', 'Narration', 'Transaction Value'])
fig.update_layout(
    title='SB distn of Transaction Values',
    yaxis_title='Transaction Value'
)
fig.show()

In [ ]:
# identify "Square" transactions and create a new column
southside_df['Narration Group'] = southside_df['Narration'].apply(lambda x: 'Square' if 'square' in x.lower() else x)

In [ ]:
# categorize transactions into Credit and Debit based on account balance changes
southside_df = southside_df.sort_values(by='Date').reset_index(drop=True)
southside_df['Balance Change'] = brisbane_df['Account Balance'].diff()
southside_df['Category'] = southside_df['Balance Change'].apply(lambda x: 'Credit' if x > 0 else 'Debit')

In [ ]:
# filter out negative values and select only expenses
expense_df = southside_df[southside_df['Category'] == 'Debit']
income_df = expense_df[expense_df['Transaction Value'] > 0]

In [ ]:
# group by 'Narration Group' and sum the 'Transaction Value'
income_breakdown = income_df.groupby('Narration Group')['Transaction Value'].sum()

In [ ]:
# get the top 10 expense categories and group the rest into 'Other'
top_expense_breakdown = income_breakdown.nlargest(10)
other_expenses = income_breakdown.sum() - top_expense_breakdown.sum()
top_expense_breakdown['Other'] = other_expenses

In [ ]:
# plot expense breakdown
top_expense_breakdown.plot(kind='pie', autopct='%1.1f%%', figsize=CFG.fig_size)
plt.title('Top 10 Expense Breakdown')
plt.ylabel('')
plt.show()

# Cairns

In [ ]:
cairns_df = pd.read_csv(
    os.path.join(CFG.data_path, 'Cairns.csv'),
    names=CFG.column_names
)
cairns_df.head(CFG.n_rows)

In [ ]:
cairns_df['Date'] = pd.to_datetime(
    cairns_df['Date'], 
    dayfirst=True, 
    errors='coerce'
)
cairns_df.dtypes

In [ ]:
cairns_df.info()

In [ ]:
cairns_df.describe()

In [ ]:
cairns_df.isnull().sum()

In [ ]:
plt.figure(figsize=CFG.fig_size)
plt.plot(cairns_df['Date'], cairns_df['Account Balance'], marker='o', linestyle='-')
plt.title('Cairns Account Balance')
plt.xlabel('Date')
plt.ylabel('Account Balance')
plt.grid(True)
plt.show()
# todo: add trend line

# Mackay

In [ ]:
mackay_df = pd.read_csv(
    os.path.join(CFG.data_path, 'Mackay.csv'),
    names=CFG.column_names
)
mackay_df.head(CFG.n_rows)

In [ ]:
mackay_df['Date'] = pd.to_datetime(
    mackay_df['Date'], 
    dayfirst=True, 
    errors='coerce'
)
mackay_df.dtypes

In [ ]:
mackay_df.info()

In [ ]:
mackay_df.describe()

In [ ]:
mackay_df.isnull().sum()

In [ ]:
plt.figure(figsize=CFG.fig_size)
plt.plot(mackay_df['Date'], mackay_df['Account Balance'], marker='o', linestyle='-')
plt.title('Mackay Account Balance')
plt.xlabel('Date')
plt.ylabel('Account Balance')
plt.grid(True)
plt.show()
# todo: add trend line

# Gold Coast

In [ ]:
gc_df = pd.read_csv(
    os.path.join(CFG.data_path, 'LilleyBuddy.csv'),
    names=CFG.column_names
)
gc_df.head(CFG.n_rows)

In [ ]:
gc_df['Date'] = pd.to_datetime(
    mackay_df['Date'], 
    dayfirst=True, 
    errors='coerce'
)
gc_df.dtypes

In [ ]:
gc_df.info()

In [ ]:
gc_df.describe()

In [ ]:
gc_df.isnull().sum()

In [ ]:
plt.figure(figsize=CFG.fig_size)
plt.plot(gc_df['Date'], gc_df['Account Balance'], marker='o', linestyle='-')
plt.title('Gold Coast Account Balance')
plt.xlabel('Date')
plt.ylabel('Account Balance')
plt.grid(True)
plt.show()
# todo: add trend line